In [1]:
import sys
import os.path
import json
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
from itertools import product
import gc
import re
from utils import downcast_dtypes

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive') 
  if not os.path.isfile('SETTINGS.json'):
       # hard coded data directory in drive is used if SETTINGS.json not present 
       config={}
       config['DATA_DIR'] = '/content/gdrive/My Drive/kaggle-c1'
       with open('SETTINGS.json', 'w') as outfile:
         json.dump(config, outfile)

with open('SETTINGS.json') as config_file:
    config = json.load(config_file)

DATA_DIR = config['DATA_DIR']

print('Using DATA_DIR ', DATA_DIR)

DATA_FOLDER = DATA_DIR

Using DATA_DIR  /home/vvi/repos/kaggle-final-project/datadir


In [2]:

sales    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_cats = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
sample_submission = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

# Textual item categories

The predictions should employ features extracted from text. This will be done, even though textual features do not seem very relevant in this particular task.

Textual features and "special text processing methods" are used by converting  
item names into TF-IDF transformed BOW-vectors. The vectors are clustered in order to get new categorization of products. Two clusterings are produced, one based unigrams and another based on bigrams. Additionally, one more textual clustering is produced based on the occurrences of the most frequent terms in item names (instead of TF-IDF weighting, which emphasizes uncommon words).

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus=items['item_name'].values
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=2)
item_name_bow=vectorizer.fit_transform(corpus)
print(item_name_bow.shape)

vectorizer_bigram = TfidfVectorizer(sublinear_tf=True, ngram_range=(2,2), min_df=2)
item_name_bow_bigram=vectorizer_bigram.fit_transform(corpus)
print(item_name_bow_bigram.shape)


(22170, 9530)
(22170, 13242)


In addition to thew TFIDF representations that emphasize the the terms distinctive to documents,
construct a representation from the most frequent words. Hand-pick from the most frequent terms 
the ones that are likely to flag relevant product attributes.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_counts=CountVectorizer(binary=True, max_features=300)
count_matrix=vectorizer_counts.fit_transform(corpus)


In [5]:
# output a file that lists most common words together with their frequencies
counts=np.ravel(np.sum(count_matrix.todense(), axis=0))
idx=np.argsort(-counts)
np.ravel(counts[idx])
df=pd.DataFrame()
df['term']=np.array(vectorizer_counts.get_feature_names())[list(idx)]
df['frequency']=np.ravel(counts[idx])
df.to_csv('frequent_item_name_terms.csv')
# from this list manually select terms to be removed -> stopwords_item_name.csv         

In [6]:
!wget -O stopwords_item_name.csv https://github.com/vvivvi/kaggle-c1/blob/master/stopwords_item_name.csv?raw=true
df_stop=pd.read_csv('stopwords_item_name.csv')
df_stop

--2020-11-23 14:34:17--  https://github.com/vvivvi/kaggle-c1/blob/master/stopwords_item_name.csv?raw=true
Selvitetään osoitetta github.com (github.com)... 140.82.121.3
Yhdistetään palvelimeen github.com (github.com)|140.82.121.3|:443... yhdistetty.
HTTP-pyyntö lähetetty, odotetaan vastausta... 302 Found
Sijainti: https://github.com/vvivvi/kaggle-c1/raw/master/stopwords_item_name.csv [seurataan]
--2020-11-23 14:34:18--  https://github.com/vvivvi/kaggle-c1/raw/master/stopwords_item_name.csv
Käytetään uudelleen yhteyttä github.com:443.
HTTP-pyyntö lähetetty, odotetaan vastausta... 302 Found
Sijainti: https://raw.githubusercontent.com/vvivvi/kaggle-c1/master/stopwords_item_name.csv [seurataan]
--2020-11-23 14:34:18--  https://raw.githubusercontent.com/vvivvi/kaggle-c1/master/stopwords_item_name.csv
Selvitetään osoitetta raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.84.133
Yhdistetään palvelimeen raw.githubusercontent.com (raw.githubusercontent.com)|151.101.84.133|:443...

,Unnamed: 0,term,frequency
0,0,версия,3519
1,7,the,1118
2,10,of,912
3,18,для,625
4,24,на,478
5,46,по,223
6,57,10,182
7,58,за,179
8,62,из,167
9,66,in,161


These are the rather arbitrarily hand-picked stopwords. We notice that their total number does not hugely mask the other frequent words. We would probably do just fine completely without stopword list. However, since we already picked it, let's use it: 

In [7]:
stopwords=list(df_stop['term'])
vectorizer_counts=CountVectorizer(binary=True, max_features=300-len(stopwords), stop_words=stopwords)
count_matrix_frequent=vectorizer_counts.fit_transform(corpus)

Now create the three new clusterings of items with the K means clustering algorithm

In [8]:
from sklearn.cluster import KMeans

kmeans_bow_256 = KMeans(n_clusters=256, random_state=123, n_jobs=-1).fit(item_name_bow)
print('.')
kmeans_bow_bigram256 = KMeans(n_clusters=256, random_state=123, n_jobs=-1).fit(item_name_bow_bigram)
print('.')
kmeans_bow_frequent256 = KMeans(n_clusters=256, random_state=123, n_jobs=-1).fit(count_matrix_frequent)
print('.')


.
.
.


In [9]:
# collect clustering results to a pandas dataframe and save to csv file
items_with_added_features=pd.DataFrame(items,copy=True)

items_with_added_features['item_name_category_tfidf_unigram_256']=kmeans_bow_256.labels_
items_with_added_features['item_name_category_tfidf_bigram_256']=kmeans_bow_bigram256.labels_
items_with_added_features['item_name_category_frequent_256']=kmeans_bow_frequent256.labels_


In [10]:
items_with_added_features.shape

(22170, 6)

# Additional item features

In [13]:
# Add a heuristic text feature from item name: cyrillic_fraction
#
# the idea behind this feature is that "international" items may have different sales statistics
# as Russian ones with names completely written with cyrillic letters

In [11]:
def cyrillic_fraction(str):
    if len(str) == 0:
        return 0
    non_cyrillic=re.sub('[\u0400-\u04FF]', '',str)
    return 1.0-len(non_cyrillic)/len(str)

In [12]:
items_with_added_features['item_name_cyrillic_fraction']=items_with_added_features['item_name'].map(cyrillic_fraction)

# all text processing done, actual text can be dropped
items_with_added_features.drop('item_name', axis=1, inplace=True)


In [13]:
items_with_added_features.shape

(22170, 6)

In [14]:
# insert item price to set of item features
items_with_added_features=pd.merge(items_with_added_features, sales.groupby('item_id')['item_price'].mean().reset_index(), how='left',on='item_id').fillna(0)

In [15]:
items_with_added_features.head()

,item_id,item_category_id,item_name_category_tfidf_unigram_256,item_name_category_tfidf_bigram_256,item_name_category_frequent_256,item_name_cyrillic_fraction,item_price
0,0,40,13,4,0,0.560976,58.0
1,1,76,80,26,99,0.205882,4490.0
2,2,40,13,4,0,0.244444,58.0
3,3,40,13,4,0,0.255319,79.0
4,4,40,13,4,0,0.302326,58.0


In [16]:
items_with_added_features.to_csv(DATA_FOLDER + '/items_with_added_features.csv')

In [17]:
items_with_added_features.shape

(22170, 7)

# Feature matrix generation

Aggregate the transactional data into grid of monthly summaries, each row indexed by (shop_id, item_id, data_block_num) triplet. Drop shop ids not in test data.

In [18]:
test_shops=test_spec['shop_id'].unique()
test_items=test_spec['item_id'].unique()

date_block_test = 35 # Dec 2015
sales = sales[sales['shop_id'].isin(test_shops)]

In [19]:
# as identified in the EDA phase, correct a mistaken shop id 11 -> 10,
# 0 -> 57, 1 -> 58
sales.loc[sales.shop_id==11, 'shop_id'] = 10
sales.loc[sales.shop_id==0, 'shop_id'] = 57
sales.loc[sales.shop_id==1, 'shop_id'] = 58



In [20]:
shop_and_item_id_to_test_id={}

sr = test_spec['item_id'].astype(str) + '_' + test_spec['shop_id'].astype(str)
for index, val in sr.iteritems():
  shop_and_item_id_to_test_id[val] = index



In [21]:
len(shop_and_item_id_to_test_id)

214200

In [22]:
# this part of the code 
# and thus the idea of lagged targets grouped by different categories 
# modified from the course notebook on predictor stacking

# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# add test month in the order specified by test_spec    
# grid.append(np.array(list(product(*[test_shops, test_items, [date_block_test]])),dtype='int32'))
test_array=np.array(test_spec[['shop_id','item_id','shop_id']], dtype='int32')
test_array[:,2]=date_block_test
grid.append(test_array)

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'}).rename(columns={'item_cnt_day':'target'})
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'}).rename(columns={'item_cnt_day':'target_shop'})
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'}).rename(columns={'item_cnt_day':'target_item'})
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
del grid, gb 
gc.collect();

In [23]:
all_data=pd.merge(all_data, items_with_added_features, on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect()


22

Aggregate 'target' by newly added item categories and item categories within particular shops


In [24]:
# first aggregate by item categories as such

category_cols = [col for col in all_data.columns if re.search('category', col)]


for column in category_cols:
  targetsuffix=column[column.find('category'):]
  targetsuffix=re.sub('_id', '', targetsuffix)
  print(column,"->",targetsuffix)  
  gb = all_data.groupby(['date_block_num',column],as_index=False).agg({'target':'sum'}).rename(columns={'target':'target_'+targetsuffix})
  all_data = pd.merge(all_data, gb, how='left', on=['date_block_num', column]).fillna(0)
  all_data = downcast_dtypes(all_data)
  gc.collect()


del gb 
gc.collect();

# then add aggregates by shop and <ADDITIONAL VARIABLE> combinations
aux_vars = category_cols + ['item_id']
for column in aux_vars:
  targetsuffix = column[column.find('category'):] if column.find('category') >=0 else column
  targetsuffix=re.sub('_id', '', targetsuffix)
  targetsuffix += '_within_shop'
  print(column,"->",targetsuffix)  
  gb = all_data.groupby(['shop_id','date_block_num',column],as_index=False).agg({'target':'sum'}).rename(columns={'target':'target_'+targetsuffix})
  all_data = pd.merge(all_data, gb, how='left', on=['shop_id','date_block_num', column]).fillna(0)
  all_data = downcast_dtypes(all_data)
  gc.collect()

del gb 
gc.collect();

item_category_id -> category
item_name_category_tfidf_unigram_256 -> category_tfidf_unigram_256
item_name_category_tfidf_bigram_256 -> category_tfidf_bigram_256
item_name_category_frequent_256 -> category_frequent_256
item_category_id -> category_within_shop
item_name_category_tfidf_unigram_256 -> category_tfidf_unigram_256_within_shop
item_name_category_tfidf_bigram_256 -> category_tfidf_bigram_256_within_shop
item_name_category_frequent_256 -> category_frequent_256_within_shop
item_id -> item_within_shop


In [25]:
# add one ad hoc binary indicator of one particular shop type (Online store) 
# discovered during EDA

internet_stores = [12,55]

all_data['is_internet_store'] = np.where((all_data['shop_id']==12) | (all_data['shop_id']==55), 1, 0)

In [26]:
all_data.to_csv(DATA_FOLDER + '/all_data_with_category_targets.csv')

In [27]:
# create shifted versions of the category-wise target columns one column at time and write to disk
# in order to keep memory consumption manageable

In [29]:
all_data=pd.read_csv(DATA_FOLDER + '/all_data_with_category_targets.csv')
# List of columns that we will use to create lags
cols_to_shift = [col for col in all_data.columns.values if re.search('target',col)] 
index_cols = ['shop_id', 'item_id', 'date_block_num']

keys_to_target= {'target' : ['shop_id', 'item_id', 'date_block_num'],
  'target_shop': ['shop_id', 'date_block_num'],
  'target_item': ['item_id', 'date_block_num'],
  'target_category': ['item_category_id', 'date_block_num'], 
  'target_category_tfidf_unigram_256': ['item_name_category_tfidf_unigram_256','date_block_num'],
  'target_category_tfidf_bigram_256': ['item_name_category_tfidf_bigram_256','date_block_num'],
  'target_category_frequent_256': ['item_name_category_frequent_256','date_block_num'],
  'target_item_within_shop': ['shop_id', 'item_id', 'date_block_num'],
  'target_category_within_shop': ['shop_id', 'item_category_id', 'date_block_num'], 
  'target_category_tfidf_unigram_256_within_shop': ['shop_id', 'item_name_category_tfidf_unigram_256','date_block_num'],
  'target_category_tfidf_bigram_256_within_shop': ['shop_id', 'item_name_category_tfidf_bigram_256','date_block_num'],
  'target_category_frequent_256_within_shop': ['shop_id', 'item_name_category_frequent_256','date_block_num']
             }

shift_range = [2, 3, 4, 5,6,12]

# create shifted versions of one column at time and write to disk
# in order to keep memory consumption manageable

for col in cols_to_shift:
    print(col)
    all_data=pd.read_csv(DATA_FOLDER + '/all_data_with_category_targets.csv')
    target_with_keys=all_data[keys_to_target[col]+[col]].copy()
    lagged_data=all_data.copy()
    lagged_data=downcast_dtypes(lagged_data)
    target_with_keys=downcast_dtypes(target_with_keys).drop_duplicates()
    del all_data
    gc.collect()
    for month_shift in shift_range:
        train_shift = target_with_keys.copy()
        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x==col else x
        train_shift = train_shift.rename(columns=foo)

        lagged_data = pd.merge(lagged_data, train_shift, on=keys_to_target[col], how='left').fillna(0)
        lagged_data=downcast_dtypes(lagged_data)

        del train_shift
        gc.collect()

        # Don't use old data without enough lags in the past
    lagged_data = lagged_data[lagged_data['date_block_num'] >= max(shift_range)] 
    to_drop_cols=[col for col in lagged_data.columns.values if not re.search('lag',col) and not col in index_cols]
    lagged_data.drop(to_drop_cols,axis=1).to_csv(DATA_FOLDER + '/' + col + '_lagged.csv')
    
    del lagged_data
    gc.collect()


target
target_shop
target_item
target_category
target_category_tfidf_unigram_256
target_category_tfidf_bigram_256
target_category_frequent_256
target_category_within_shop
target_category_tfidf_unigram_256_within_shop
target_category_tfidf_bigram_256_within_shop
target_category_frequent_256_within_shop
target_item_within_shop


# Feature set generation

Now that all the features have been written onto disk, feature sets can be selected out of them. For this project, we generate five 
tentative sets of features to act as basis for further feature selection:

1. Basic feature set: miscellanous non-lagged features and temporally lagged versions of monthly {target,item,shop} -wise sales
2. Textual feature set: item categories based on item name + temporally lagged sales grouped by textual categories
3. Shopwise feature set: temporally lagged sales grouped by non-textual and textual categories for each shop separately
4. Basic features 1. augmented with shopwise features 3.. Denoted basicv2
5. All the generated features

These feature subsets have been chosen heuristically and are rather arbitrary.


In [30]:
# first, list all the generated feature files
feature_files = [
  'all_data_with_category_targets.csv',
  'target_category_frequent_256_lagged.csv',
  'target_category_frequent_256_within_shop_lagged.csv',
  'target_category_lagged.csv',
  'target_category_tfidf_bigram_256_lagged.csv',
  'target_category_tfidf_bigram_256_within_shop_lagged.csv',
  'target_category_tfidf_unigram_256_lagged.csv',
  'target_category_tfidf_unigram_256_within_shop_lagged.csv',
  'target_category_within_shop_lagged.csv',
  'target_item_lagged.csv',
  'target_lagged.csv',
  'target_shop_lagged.csv',
]               

In [31]:
feature2filename={}

# create data structure that tells in which feature file a given feature is found

for f in feature_files:
    df =pd.read_csv(DATA_FOLDER + '/' + f)
    for col in df.columns.values:
        feature2filename[col]=f

In [33]:
# check if all the expected features are still present: list all features
sorted(feature2filename.keys())

['Unnamed: 0',
 'date_block_num',
 'is_internet_store',
 'item_category_id',
 'item_id',
 'item_name_category_frequent_256',
 'item_name_category_tfidf_bigram_256',
 'item_name_category_tfidf_unigram_256',
 'item_name_cyrillic_fraction',
 'item_price',
 'shop_id',
 'target',
 'target_category',
 'target_category_frequent_256',
 'target_category_frequent_256_lag_12',
 'target_category_frequent_256_lag_2',
 'target_category_frequent_256_lag_3',
 'target_category_frequent_256_lag_4',
 'target_category_frequent_256_lag_5',
 'target_category_frequent_256_lag_6',
 'target_category_frequent_256_within_shop',
 'target_category_frequent_256_within_shop_lag_12',
 'target_category_frequent_256_within_shop_lag_2',
 'target_category_frequent_256_within_shop_lag_3',
 'target_category_frequent_256_within_shop_lag_4',
 'target_category_frequent_256_within_shop_lag_5',
 'target_category_frequent_256_within_shop_lag_6',
 'target_category_lag_12',
 'target_category_lag_2',
 'target_category_lag_3',
 'tar

In [34]:
def select_features(feature_lists,filenames):
  # helper function for extracting a specified set of named feature columns
  # from a set of feature files indexed by the feature2filename structure      
  # feature_lists : list of feature lists (= feature combinations) to be red into memory 
  # simultaneously and then evaluated in one go
    index_cols = ['shop_id', 'item_id', 'date_block_num']    
    
    files_and_columns_to_read={}
    lagged=False
    
    flat_features = set([item for sublist in feature_lists for item in sublist])
    
    for feat in flat_features.union({'target'}):
        is_index=False
        for idx in index_cols:
            if feat == idx:
                is_index = True
        # index columns are present in every feature file, no need to read
        # them explicitly
        
        if is_index: 
            continue
            
        if re.search('lag', feat):
            lagged=True
        file = feature2filename[feat]
        featlist=files_and_columns_to_read.get(file,[])
        featlist += [feat]
        files_and_columns_to_read[file] = featlist
    print('file -> columns mapping: ', files_and_columns_to_read)   
    
    first=True
    for file in files_and_columns_to_read:
        to_read_cols = files_and_columns_to_read[file]
        to_keep_cols = list(set(to_read_cols + index_cols)) # add index columns, remove duplicates
        
        if first:
            all_data = pd.read_csv(DATA_FOLDER + '/' + file )[to_keep_cols]
            first=False
        else:
            df = pd.read_csv(DATA_FOLDER + '/' + file )[to_keep_cols]
            df = downcast_dtypes(df)
            all_data = pd.merge(all_data, df, on=index_cols)
            del df
            gc.collect()
        if lagged:
            all_data=all_data[all_data['date_block_num'] >= 12]
        all_data = downcast_dtypes(all_data)
        gc.collect()
                 
    print('read columns:', all_data.columns.values)
              
    for filename,features in zip(filenames,feature_lists):    
        print("Generating {}".format(filename))
        all_data[list(set(features+index_cols+['target']))].to_csv(DATA_FOLDER + '/' + filename, index=False)
        
    

In [35]:
# select the five mentioned feature subsets from all the available feature columns and write to disk

# set 1: basic features
feature_list=[f for f in sorted(feature2filename.keys()) if re.search('^target_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('^target_shop_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('^target_item_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('^target_category_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('item_name',f) ]
feature_list += ['item_category_id', 'item_price', 'is_internet_store']
lists=[feature_list]

# set 2: textual features
feature_list= [f for f in sorted(feature2filename.keys()) if re.search('item_name',f) ]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('tfidf.*lag_',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('frequent.*lag_',f)]
feature_list += ['item_category_id']
lists.append(feature_list)

# set 3: lagged features within shop
feature_list=[f for f in sorted(feature2filename.keys()) if re.search('within.*lag_',f)]
feature_list += ['item_category_id']
lists.append(feature_list)

# set 4: basic feature set v2
feature_list=[f for f in sorted(feature2filename.keys()) if re.search('^target_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('^target_shop_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('^target_item_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('^target_category_lag_(\d+)',f)]
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('item_name',f) ]
feature_list += ['item_category_id', 'item_price','is_internet_store']
feature_list += [f for f in sorted(feature2filename.keys()) if re.search('within.*lag_',f)]
lists.append(list(set(feature_list)))

#set 5: all the calculated features

feature_list = list(set(lists[0] + lists[1] + lists[2]))
lists.append(list(set(feature_list)))

select_features(lists, ['feature_set_basic.csv','feature_set_text.csv','feature_set_within.csv','feature_set_basicv2.csv', 'feature_set_allfeat.csv'])


file -> columns mapping:  {'target_category_tfidf_bigram_256_within_shop_lagged.csv': ['target_category_tfidf_bigram_256_within_shop_lag_12', 'target_category_tfidf_bigram_256_within_shop_lag_4', 'target_category_tfidf_bigram_256_within_shop_lag_6', 'target_category_tfidf_bigram_256_within_shop_lag_5', 'target_category_tfidf_bigram_256_within_shop_lag_3', 'target_category_tfidf_bigram_256_within_shop_lag_2'], 'target_category_within_shop_lagged.csv': ['target_category_within_shop_lag_4', 'target_category_within_shop_lag_12', 'target_category_within_shop_lag_6', 'target_category_within_shop_lag_2', 'target_category_within_shop_lag_3', 'target_category_within_shop_lag_5'], 'target_item_lagged.csv': ['target_item_lag_4', 'target_item_lag_2', 'target_item_lag_5', 'target_item_lag_3', 'target_item_lag_6', 'target_item_lag_12'], 'target_shop_lagged.csv': ['target_shop_lag_6', 'target_shop_lag_12', 'target_shop_lag_3', 'target_shop_lag_4', 'target_shop_lag_5', 'target_shop_lag_2'], 'target_ca